In [1]:
from pymongo import MongoClient
from pprint import pprint

In [22]:
# Replace with your mongos address and authentication if needed
mongos_uri = "mongodb://192.168.178.168:32017"

# Connect to the mongos router
client = MongoClient(mongos_uri)

# Test connection by listing databases
try:
    print("Connected to MongoDB. Databases:")
    pprint(client.list_database_names())
except Exception as e:
    print("Connection failed:", e)


Connected to MongoDB. Databases:
['admin', 'config', 'ycsb_sharded']


In [30]:
def list_shards(mongos_client: MongoClient):
    """
    Lists all current shards in the MongoDB sharded cluster.
    
    :param mongos_client: The MongoClient connected to mongos
    """
    try:
        result = mongos_client.admin.command("listShards")
        print("📦 Current shards in the cluster:")
        for shard in result["shards"]:
            print(f" - {shard['_id']}: {shard['host']}")
    except Exception as e:
        print(f"❌ Failed to list shards: {e}")

# List current shards
list_shards(client)

📦 Current shards in the cluster:
 - shard1: shard1/mongodb-shard1-0.mongodb-shard1.default.svc.cluster.local:27018
 - shard2: shard2/mongodb-shard2-0.mongodb-shard2.default.svc.cluster.local:27018
 - shard3: shard3/mongodb-shard3-0.mongodb-shard3.default.svc.cluster.local:27018
 - shard4: shard4/mongodb-shard4-0.mongodb-shard4.default.svc.cluster.local:27018


In [57]:
def remove_shard(mongos_client: MongoClient, shard_id: int):
    """
    Initiates or continues removal of a shard from the MongoDB cluster.
    This function executes the `removeShard` command only once, allowing
    external control over when to reissue it.

    :param mongos_client: The MongoClient connected to mongos
    :param shard_id: Numeric string like '3' (for shard3)
    """
    shard_name = f"shard{shard_id}"

    try:
        res = mongos_client.admin.command("removeShard", shard_name)
        state = res["state"]
        print(f"📦 removeShard called on: {shard_name} (state: {state})")
        pprint(res)
    except Exception as e:
        print(f"❌ Failed to remove shard {shard_name}: {e}")


def add_shard(mongos_client: MongoClient, shard_id: int):
    """
    Adds a shard to the MongoDB cluster. Constructs shard URI from the shard ID.
    
    :param mongos_client: The MongoClient connected to mongos
    :param shard_id: Numeric string like '3' (for shard3)
    """
    shard_name = f"shard{shard_id}"
    host = f"mongodb-shard{shard_id}-0.mongodb-shard{shard_id}.default.svc.cluster.local:27018"
    shard_uri = f"{shard_name}/{host}"

    try:
        result = mongos_client.admin.command("addShard", shard_uri)
        print(f"✅ Shard added: {shard_uri}")
        pprint(result)
    except Exception as e:
        print(f"❌ Failed to add shard {shard_uri}: {e}")


def reshard_collection(mongos_client: MongoClient, db_name="ycsb_sharded", coll_name="usertable", chunks=40):
    """
    Initiates a reshardCollection operation on the given collection with:
    - forceRedistribution: true
    - numInitialChunks: 40
    - shardKey: { country: "hashed" }

    :param mongos_client: MongoClient connected to mongos
    :param db_name: Name of the database (e.g., "ycsb_sharded")
    :param coll_name: Name of the collection (e.g., "usertable")
    """
    try:
        namespace = f"{db_name}.{coll_name}"
        cmd = {
            "reshardCollection": namespace,
            "key": { "country": "hashed" },
            "numInitialChunks": chunks,
            "forceRedistribution": True
        }
        print(f"✅ Resharding initiated for {namespace} with {chunks} chunks")
        result = mongos_client.admin.command(cmd)
        from pprint import pprint
        pprint(result)

    except Exception as e:
        print(f"❌ Failed to reshard {namespace}: \n{e}")


import hashlib
from bson.int64 import Int64

def split_hashed_chunks(mongos_client: MongoClient, db_name="ycsb_sharded", coll_name="usertable", num_chunks=40):
    """
    Splits a collection sharded on a hashed key into `num_chunks` chunks
    using evenly distributed hashed key space split points.

    :param mongos_client: MongoClient connected to mongos
    :param db_name: Name of the database
    :param coll_name: Name of the collection
    :param num_chunks: Number of chunks to split into (default: 40)
    """
    try:
        print(f"🔧 Splitting {db_name}.{coll_name} into {num_chunks} chunks on hashed 'country'...")

        ns = f"{db_name}.{coll_name}"
        min_hash = -2**63
        max_hash = 2**63

        step = (max_hash - min_hash) // num_chunks
        split_points = [Int64(min_hash + i * step) for i in range(1, num_chunks)]

        for i, split_point in enumerate(split_points):
            cmd = {
                "split": ns,
                "middle": { "country": split_point }
            }
            res = mongos_client.admin.command(cmd)
            print(f"✅ Split {i+1}/{num_chunks - 1} at hashed value {split_point}")

        print(f"🎉 Done splitting into {num_chunks} chunks.")

    except Exception as e:
        print(f"❌ Failed during split: {e}")


In [29]:
add_shard(client, 4)

✅ Shard added: shard4/mongodb-shard4-0.mongodb-shard4.default.svc.cluster.local:27018
{'$clusterTime': {'clusterTime': Timestamp(1747061550, 18),
                  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00'
                                        b'\x00\x00\x00\x00\x00\x00\x00\x00'
                                        b'\x00\x00\x00\x00',
                                'keyId': 0}},
 'ok': 1.0,
 'operationTime': Timestamp(1747061550, 18),
 'shardAdded': 'shard4'}


In [58]:
remove_shard(client, 4)

📦 removeShard called on: shard4 (state: started)
{'$clusterTime': {'clusterTime': Timestamp(1747063790, 3),
                  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00'
                                        b'\x00\x00\x00\x00\x00\x00\x00\x00'
                                        b'\x00\x00\x00\x00',
                                'keyId': 0}},
 'dbsToMove': [],
 'msg': 'draining started successfully',
 'note': 'you need to drop or movePrimary these databases',
 'ok': 1.0,
 'operationTime': Timestamp(1747063790, 3),
 'shard': 'shard4',
 'state': 'started'}


In [61]:
reshard_collection(client, chunks=26)

✅ Resharding initiated for ycsb_sharded.usertable with 26 chunks
{'$clusterTime': {'clusterTime': Timestamp(1747064881, 81),
                  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00'
                                        b'\x00\x00\x00\x00\x00\x00\x00\x00'
                                        b'\x00\x00\x00\x00',
                                'keyId': 0}},
 'ok': 1.0,
 'operationTime': Timestamp(1747064881, 81)}


In [45]:
split_hashed_chunks(client, num_chunks=40)

🔧 Splitting ycsb_sharded.usertable into 40 chunks on hashed 'country'...
✅ Split 1/39 at hashed value -8762203435012037018
✅ Split 2/39 at hashed value -8301034833169298228
✅ Split 3/39 at hashed value -7839866231326559438
✅ Split 4/39 at hashed value -7378697629483820648
✅ Split 5/39 at hashed value -6917529027641081858
✅ Split 6/39 at hashed value -6456360425798343068
✅ Split 7/39 at hashed value -5995191823955604278
✅ Split 8/39 at hashed value -5534023222112865488
✅ Split 9/39 at hashed value -5072854620270126698
✅ Split 10/39 at hashed value -4611686018427387908
✅ Split 11/39 at hashed value -4150517416584649118
✅ Split 12/39 at hashed value -3689348814741910328
✅ Split 13/39 at hashed value -3228180212899171538
✅ Split 14/39 at hashed value -2767011611056432748
✅ Split 15/39 at hashed value -2305843009213693958
✅ Split 16/39 at hashed value -1844674407370955168
✅ Split 17/39 at hashed value -1383505805528216378
✅ Split 18/39 at hashed value -922337203685477588
✅ Split 19/39 at ha